In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS

In [96]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_United_States_hurricanes")
page = BS(r.text)

In [97]:
rows = []
for t in page.find_all("table"):
    th = t.find("th")
    if th == None:
        continue
    if th.get_text().strip() != "Name":
        continue
    for tr in t.find_all("tr"):
        td = tr.find("td")
        if td == None:
            continue
        a = td.find("a")
        name = td.get_text().strip()
        length = len(name.split(" "))
        if a == None or name == "Unnamed" or name == "Name":
            continue
        url = a["href"]
        if not url.startswith("/wiki/"):
            continue
        rows.append({"name":name, "href":"https://en.wikipedia.org" + url})
print(rows[:10])

[{'name': 'Baker', 'href': 'https://en.wikipedia.org/wiki/Hurricane_Baker_(1950)'}, {'name': 'Camille', 'href': 'https://en.wikipedia.org/wiki/Hurricane_Camille'}, {'name': 'Eloise', 'href': 'https://en.wikipedia.org/wiki/Hurricane_Eloise'}, {'name': 'Frederic', 'href': 'https://en.wikipedia.org/wiki/Hurricane_Frederic'}, {'name': 'Elena', 'href': 'https://en.wikipedia.org/wiki/Hurricane_Elena'}, {'name': 'Opal', 'href': 'https://en.wikipedia.org/wiki/Hurricane_Opal'}, {'name': 'Danny', 'href': 'https://en.wikipedia.org/wiki/Hurricane_Danny_(1997)'}, {'name': 'Ivan', 'href': 'https://en.wikipedia.org/wiki/Hurricane_Ivan'}, {'name': 'Dennis', 'href': 'https://en.wikipedia.org/wiki/Hurricane_Dennis'}, {'name': 'Katrina', 'href': 'https://en.wikipedia.org/wiki/Hurricane_Katrina'}]


In [ ]:
pages = {}

In [60]:
for row in rows:
    print(row["href"])
    if row["href"] in pages:
        continue
    r = requests.get(row["href"])
    pages[row["href"]] = BS(r.text)

https://en.wikipedia.org/wiki/Hurricane_Baker_(1950)
https://en.wikipedia.org/wiki/Hurricane_Camille
https://en.wikipedia.org/wiki/Hurricane_Eloise
https://en.wikipedia.org/wiki/Hurricane_Frederic
https://en.wikipedia.org/wiki/Hurricane_Elena
https://en.wikipedia.org/wiki/Hurricane_Opal
https://en.wikipedia.org/wiki/Hurricane_Danny_(1997)
https://en.wikipedia.org/wiki/Hurricane_Ivan
https://en.wikipedia.org/wiki/Hurricane_Dennis
https://en.wikipedia.org/wiki/Hurricane_Katrina
https://en.wikipedia.org/wiki/Hurricane_Michael
https://en.wikipedia.org/wiki/Hurricane_Carol
https://en.wikipedia.org/wiki/Hurricane_Donna
https://en.wikipedia.org/wiki/Hurricane_Gloria
https://en.wikipedia.org/wiki/Hurricane_Bob
https://en.wikipedia.org/wiki/Labor_Day_Hurricane_of_1935
https://en.wikipedia.org/wiki/Hurricane_Easy_(1950)
https://en.wikipedia.org/wiki/Hurricane_King
https://en.wikipedia.org/wiki/Hurricane_Florence_(1953)
https://en.wikipedia.org/wiki/Hurricane_Hazel_(1953)
https://en.wikipedia.org

https://en.wikipedia.org/wiki/Hurricane_Iwa
https://en.wikipedia.org/wiki/Hurricane_Iniki
https://en.wikipedia.org/wiki/Cyclone_Tusi
https://en.wikipedia.org/wiki/Cyclone_Ofa
https://en.wikipedia.org/wiki/Cyclone_Val
https://en.wikipedia.org/wiki/Cyclone_Heta
https://en.wikipedia.org/wiki/Cyclone_Olaf
https://en.wikipedia.org/wiki/2009%E2%80%9310_South_Pacific_cyclone_season#Severe_Tropical_Cyclone_Rene
https://en.wikipedia.org/wiki/1949_Pacific_typhoon_season#Typhoon_Allyn
https://en.wikipedia.org/wiki/1957_Pacific_typhoon_season#Typhoon_Lola
https://en.wikipedia.org/wiki/Typhoon_Karen_(1962)
https://en.wikipedia.org/wiki/1963_Pacific_typhoon_season#Typhoon_Olive
https://en.wikipedia.org/wiki/1964_Pacific_typhoon_season#Super_Typhoon_Sally_(Aring)
https://en.wikipedia.org/wiki/1975_Pacific_typhoon_season#Super_Typhoon_June_(Rosing)
https://en.wikipedia.org/wiki/Typhoon_Pamela_(1976)
https://en.wikipedia.org/wiki/Typhoon_Rita_(1978)
https://en.wikipedia.org/wiki/1984_Pacific_typhoon_se

In [164]:
import re
rows2 = []
hrefs = set()
for row in rows:
    if row["href"] in hrefs:
        continue
    hrefs.add(row["href"])
    page = pages[row["href"]]
    fields = {}
    for tr in page.find_all("tr"):
        tds = tr.find_all(["td", "th"])
        tds = [td.get_text().strip().lower() for td in tds]
        if len(tds) != 2:
            continue
        fields[tds[0]] = tds[1].strip().lower().replace(",", "")
    try:
        if " " in row["name"]:
            continue
        if not fields["damage"].startswith("$"):
            continue
        damage = fields["damage"].lstrip("$").split(" ")
        if damage[1] == "million":
            row["damage"] = int(float(damage[0]) * 1e6)
        elif damage[1] == "billion":
            row["damage"] = int(float(damage[0]) * 1e9)
        else:
            row["damage"] = int(float(damage[0]))
        row["deaths"] = int(fields["fatalities"].split(" ")[0])
        row["year"] = fields["formed"].split("(")[0].strip()[-4:]
        m = re.match(r'.*?(\d+) mph', fields["highest winds"])
        row["mph"] = int(m.group(1))
        rows2.append(row)
    except Exception as e:
        if row["name"] == "Katrina":
            raise e
rows2[:5]

[{'name': 'Baker',
  'href': 'https://en.wikipedia.org/wiki/Hurricane_Baker_(1950)',
  'damage': 2550000,
  'fatalities': 38,
  'year': '1950',
  'mph': 120,
  'deaths': 38},
 {'name': 'Camille',
  'href': 'https://en.wikipedia.org/wiki/Hurricane_Camille',
  'damage': 1430000000,
  'fatalities': 259,
  'year': '1969',
  'mph': 175,
  'deaths': 259},
 {'name': 'Eloise',
  'href': 'https://en.wikipedia.org/wiki/Hurricane_Eloise',
  'damage': 560000000,
  'fatalities': 80,
  'year': '1975',
  'mph': 125,
  'deaths': 80},
 {'name': 'Frederic',
  'href': 'https://en.wikipedia.org/wiki/Hurricane_Frederic',
  'damage': 1770000000,
  'fatalities': 12,
  'year': '1979',
  'mph': 130,
  'deaths': 12},
 {'name': 'Elena',
  'href': 'https://en.wikipedia.org/wiki/Hurricane_Elena',
  'damage': 1300000000,
  'fatalities': 9,
  'year': '1985',
  'mph': 125,
  'deaths': 9}]

In [173]:
print(len(rows2))
df = pd.DataFrame(rows2)
df.head()

111


,damage,deaths,fatalities,href,mph,name,year
0,2550000,38,38,https://en.wikipedia.org/wiki/Hurricane_Baker_...,120,Baker,1950
1,1430000000,259,259,https://en.wikipedia.org/wiki/Hurricane_Camille,175,Camille,1969
2,560000000,80,80,https://en.wikipedia.org/wiki/Hurricane_Eloise,125,Eloise,1975
3,1770000000,12,12,https://en.wikipedia.org/wiki/Hurricane_Frederic,130,Frederic,1979
4,1300000000,9,9,https://en.wikipedia.org/wiki/Hurricane_Elena,125,Elena,1985


In [174]:
df = df[["name", "year", "mph", "damage", "deaths"]]


In [175]:
df.to_csv("hurricanes.csv", index=False)